In [2]:
import cv2
import numpy as np
import os
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Step 1: Collect images and labels from the dataset folder
def collect_images(image_folder, image_size=(100, 100)):
    images = []
    labels = []
    label_map = {}  # To map label to integer value
    
    label_counter = 0  # Start assigning label IDs from 0
    for filename in os.listdir(image_folder):
        if filename.endswith(".jpg"):
            # Extract the label (name part before the underscore)
            label_name = filename.split('_')[0]
            
            if label_name not in label_map:
                label_map[label_name] = label_counter
                label_counter += 1
                
            # Read and process the image
            img = cv2.imread(os.path.join(image_folder, filename), cv2.IMREAD_GRAYSCALE)
            if img is not None:
                img_resized = cv2.resize(img, image_size)  # Resize all images to a consistent size
                images.append(img_resized.flatten())  # Flatten the image to 1D vector
                labels.append(label_name)  # Append the label name directly
    
    return np.array(images), labels

# Step 2: Train the PCA model on the images
def train_pca_model(images):
    scaler = StandardScaler()
    images = scaler.fit_transform(images)  # Standardize the data
    pca = PCA(n_components=50)  # Use 50 principal components
    pca_images = pca.fit_transform(images)  # Apply PCA
    return pca, pca_images, scaler

# Step 3: Face detection in the video stream
def detect_face_in_video(image_folder, image_size=(100, 100)):
    # Load the pre-trained Haar Cascade face detector
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    
    # Start video capture
    cap = cv2.VideoCapture("video.mp4")
    
    # Collect images and train PCA model
    images, labels = collect_images(image_folder, image_size)
    pca, pca_images, scaler = train_pca_model(images)
    
    while True:
        # Capture frame-by-frame
        ret, frame = cap.read()
        if not ret:
            break
        
        # Convert the frame to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # Detect faces in the frame
        faces = face_cascade.detectMultiScale(gray, 1.1, 4)
        
        for (x, y, w, h) in faces:
            # Crop the face region from the image
            face = gray[y:y+h, x:x+w]
            face_resized = cv2.resize(face, image_size)  # Resize to the same size used during training
            face_vector = face_resized.flatten()  # Flatten the face to a 1D vector
            
            # Standardize and apply PCA transformation
            face_vector_scaled = scaler.transform([face_vector])
            pca_face = pca.transform(face_vector_scaled)
            
            # Match the face using PCA result
            distances = np.linalg.norm(pca_images - pca_face, axis=1)
            
            # Find the closest match based on the PCA distance
            min_distance_idx = np.argmin(distances)
            
            if np.min(distances) < 50:  # Threshold for recognition
                recognized_label = labels[min_distance_idx]  # Directly get the label
                print(recognized_label)
                cv2.putText(frame, f"Person: {recognized_label}", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Draw rectangle around the face
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
        
        # Show the frame with detected faces
        cv2.imshow("Face Recognition", frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    # Path to your image folder
    image_folder = "Faces"  # Update this with the correct folder path
    detect_face_in_video(image_folder)


Hrithik Roshan
Hrithik Roshan
Hrithik Roshan
Hrithik Roshan
Hrithik Roshan
Hrithik Roshan
Hrithik Roshan
